In [21]:
import numpy as np
import cv2
import pandas as pd
import os
import pytesseract
from PIL import Image
import time
import concurrent.futures
pd.set_option('display.max_colwidth', None)
from collections import Counter
import json

# Rerun data with new dataframe that has prefilled columns with every hold on it
# Column has paranthesis and string markers

#Todo:

#Make sure total works
#Max distance between hand & start & finish holds
#Average Distance between hand & start & finish holds
#Number of holds in each quadrant, 5x3 quadrant. 
#Get middle coordinate of each quadrant and use existing find_id method to find the closest center of quadrant.
#could manually get the coordinates of each quadrant

In [22]:
# start1 = time.time()
# end1 = time.time()
# print ("grade elapsed:", end1 - start1)

In [23]:
def find_name(filepath):
    """This function reads the image, and returns the name of the boulder"""
    
    image = Image.open(filepath)
    bounding_box = (50,127,731,155) #name location
    cropped_image = image.crop(bounding_box)
    text = pytesseract.image_to_string(cropped_image)
    text = text[:-2]
    text = text.replace("@","") #Some Emojis come as @
    
    return text

In [24]:
def find_grade(filepath):
    """This function reads the image, and returns the grade of the boulder"""
    image = Image.open(filepath)
    bounding_box = (330,155,460,175) #grade location
    cropped_image = image.crop(bounding_box)
    text = pytesseract.image_to_string(cropped_image)


    text = text.partition("V") 
    text = text[2]
    text = text.strip()
    text = text.partition(" ") 
    text = text[0]
    text = text.replace("O","0") #V0's are read as VO
    
    return text

In [25]:
def make_hold_grid():
    hand_x = [54,114,173,233,293,352,412,471,530,590,650]
    hand_y = [176,235,295,354,414,473,533,592,652,711,771,831,890,950,1009,1069,1128,1188,1247]
    foot_left_x = [18,137,256,375,494,613]
    foot_left_y = [383,502,621,740,859,978,1097]
    foot_right_x = [77,196,315,434,554,672]
    foot_right_y = [323,442,561,680,799,918,1037,1156]
    foot_bottom_x = [17,77,137,196,256,315,375,434,494,553,613,672]
    foot_bottom_y = [1276]
    
    hands = make_hold_array(hand_x,hand_y)
    foot_left = make_hold_array(foot_left_x,foot_left_y)
    foot_right = make_hold_array(foot_right_x,foot_right_y)
    foot_bottom = make_hold_array(foot_bottom_x,foot_bottom_y)
    
    holds = np.append(hands,foot_left,axis=0)
    holds = np.append(holds,foot_right,axis=0)
    holds = np.append(holds,foot_bottom,axis=0)
    
    #sort holds, y value first then x, so we sort by row, then column
    holds = holds[np.lexsort((holds[:,0],holds[:,1]))]

    return holds

In [26]:
def make_hold_array(x_coordinates, y_coordinates):
    grid = np.meshgrid(x_coordinates,y_coordinates)
    grid = np.array(grid)
    holds = grid.reshape(2,-1).T
    
    return holds

In [27]:
def make_quadrants(num_x=3,num_y=5):
    hand_x = [54,114,173,233,293,352,412,471,530,590,650]
    hand_y = [176,235,295,354,414,473,533,592,652,711,771,831,890,950,1009,1069,1128,1188,1247]
    #center of quadrants remember 3 quadrants in x direction, 5 quadrants in y direction
    num_x_quadrant = num_x
    num_y_quadrant = num_y

    hand_x_spacing = int((hand_x[-1]-hand_x[0])/(num_x_quadrant*2))
    hand_x_mid = []

    count = 0
    for x in range(hand_x[0],hand_x[-1],hand_x_spacing):
        if count % 2 != 0:
            hand_x_mid.append(x)
        count+=1

    hand_y_spacing = int((hand_y[-1]-hand_y[0])/(num_y_quadrant*2))
    hand_y_mid = []

    count = 0
    for x in range(hand_y[0],hand_y[-1],hand_y_spacing):
        if count % 2 != 0:
            hand_y_mid.append(x)
        count+=1

    hand_coordinates = make_hold_array(hand_x_mid,hand_y_mid)

    return hand_coordinates

In [28]:
def invert_element(key, value):
    """This function inverts data that looks like key:[value] to value[0]: key, value[1]: key...."""
    return {x: key for x in value}

In [29]:
def convert_dataframe(temp_data,name,grade,filepath,num_holds,hand,foot,start,finish,dist_start_finish):
    """This function creates a dataframe in the format of hold number as columns and hold type (start, foot, hand, finish)
    as the value, out of a dictionary and a grade. Dataframe is made with grade and name, which are given"""
    
    campus = 'campus' in name.lower() and foot == 0 #If campus is in name and there are no feet
    
    inverted_data = {}
    
    data = {'name':name,
            'grade':grade,
            'holdcount':num_holds,
            'filepath':filepath,
            'handholds':hand,
            'footholds':foot,
            'startholds':start,
            'finishholds':finish,
            'Distance of Climb':dist_start_finish,
            'campus':campus 
            }
    
    for key,value in temp_data.items():
        inverted_data.update(invert_element(key,value))
#         data.update(invert_element(key,value))
    data.update({f"Hold {key}":value for key,value in inverted_data.items()})
    
    return pd.DataFrame([data])

In [30]:
def processHoldType(img,hold_type,threshold):
    """This function does all the processing. Taking an image, hold type, and threshold and returning a list of(x,y) 
    coordinates for holds that match the hold type"""
    
    hold_length = 60  
    hold_height = 60  
    
    hold = cv2.imread(f"{hold_type}.png")
    hold_result = cv2.matchTemplate(img, hold, cv2.TM_CCOEFF_NORMED)
    hold_yloc, hold_xloc = np.where(hold_result >= threshold)
    hold_rectangles = [[x, y, hold_length, hold_height] for x,y in zip(hold_xloc, hold_yloc)]
    hold_rectangles, _ = cv2.groupRectangles(hold_rectangles, 1, 0.2)
    hold_rectangles = [[x, y] for x,y,_,_ in hold_rectangles]
    
    return np.array(hold_rectangles)

In [31]:
def make_clickable(val):
    # target _blank to open new window
    return '<a target="_blank" href="{}">{}</a>'.format(val, val)

In [32]:
with open('hold_types.json', 'r') as f:
    hold_type_dict = json.load(f)
    
def count_hold_types(indexed_holds,hold_type_dict = hold_type_dict):
    nested_list = [hold_type_dict[str(x)] for x in indexed_holds]

    c = Counter()
    for x in nested_list:
        c.update(x)
        
    return dict(c)

In [33]:
def count_quadrants(quadrant_holds):
    quadrant_holds = sum(quadrant_holds,[])
    c = Counter()
    c.update(quadrant_holds)
    return dict(c)

In [34]:
count = 0#TEMP
def processImage(filepath):
    """This function takes a single image's file path, processes the image, and returns a dataframe, with the format hold number for columns
    and hold type as the value."""
    global count#TEMP
    print(count)#TEMP
    count +=1#TEMP
    
    img = cv2.imread(filepath)

    hand_result = processHoldType(img,'hand',.40)
    foot_result = processHoldType(img,'foot',.40)
    start_result = processHoldType(img,'start',.60)
    finish_result = processHoldType(img,'finish',.60)
    
    hands = len(hand_result)
    foot = len(foot_result)
    start = len(start_result)
    finish = len(finish_result)
    
    num_holds = hands+foot+start+finish
    
    start_average = np.average(start_result, axis=0) 
    finish_average = np.average(finish_result, axis=0)
    
    dist_start_finish = find_distance(start_average, finish_average)
    
    matches = {2: hand_result,
          3: foot_result,
          1: start_result,
          4: finish_result,
    }
    
    matches_indexes = ['None','Start','Hand','Foot','Finish']
    
#     matches = {"hand": hand_result,
#           "foot": foot_result,
#           "start": start_result,
#           "finish": finish_result,
#     }

    found_holds = {key: find_many_id(value) for key, value in matches.items()}
    holds_counter = {f"{matches_indexes[key]}": count_hold_types(value) for key,value in found_holds.items()}
    
    all_holds = sum(found_holds.values(), [])
    holds_counter.update({'total': count_hold_types(all_holds) })
    counter_dataframe = pd.json_normalize(holds_counter, sep = "_")
    
    quadrant_grid = make_quadrants()
    quadrant_holds = [find_many_id(value,quadrant_grid) for key, value in matches.items()]
    quadrant_holds = count_quadrants(quadrant_holds)
    quadrant_holds = {f"Quadrant {key}": value for key,value in quadrant_holds.items()}
    quadrant_dataframe = pd.json_normalize(quadrant_holds, sep = "_")

    #Now we've got 4 seperate lists of hold quadrants.
    
    name = find_name(filepath)
    grade = find_grade(filepath)
    
    found_dataframe = convert_dataframe(found_holds,name,grade,filepath,num_holds,hands,foot,start,finish,dist_start_finish) 
    
    return pd.concat([found_dataframe, counter_dataframe,quadrant_dataframe], axis = 1)


In [35]:
holds = make_hold_grid()

def find_id(input_cord, holds = holds):
    """Given a coordinate (x,y) This function finds the closest hold out of a list of known holds, and returns the hold
    number"""
    
    x_dist = (input_cord[0] - holds[:, 0]) #All rows, 0th element in each row
    y_dist = (input_cord[1] - holds[:, 1]) #All rows, first element in each row
    distance = (x_dist**2) + (y_dist**2) #diagonal distance
    return np.argmin(distance)

def find_many_id(my_holds, holds = holds):
    """Given a list of hold coordinates, this function will return a list of hold numbers associated with the coordinates"""
    
    return [find_id(my_holds[i],holds) for i in range(len(my_holds))]

In [36]:
def find_distance(coordinates1, coordinates2):
    return np.linalg.norm(coordinates1 - coordinates2)

In [37]:
path = 'somephotos/'
# path = 'somephotos_split/'
images = [path+file for file in os.listdir(path)]
# images = images[:50]

start1 = time.time()

my_data = pd.DataFrame(columns=[f"Hold {item}" for item in range(0, 310)])
hold_data = pd.DataFrame(columns = [[f"{hold}_{classification}" for hold in ['Start', 'Finish', 'Hand', 'Foot'] for classification in ['Jug','Semi-Jug','Crimp','Pinch','Sloper','Undercling','Sidepull','Foot']]][0])


my_data = pd.concat([my_data,hold_data])
# my_data = pd.DataFrame()


if __name__ == "__main__":
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(processImage,images)

# for result in results:
#     my_data = pd.concat([my_data,result])
    
# # my_data.fillna('-',inplace = True) #Turn all blanks into '-' for better readability
# my_data.fillna('0',inplace = True)
# my_data.reset_index(inplace = True, drop=True)
# # my_data.sort_index(axis=1) #Transpose?

# end1 = time.time()


# print ("Time elapsed:", end1 - start1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [38]:
#For some reason breaks first time you run this code block but then works fine the second time.
for result in results:
    my_data = pd.concat([my_data,result])
    
# my_data.fillna('-',inplace = True) #Turn all blanks into '-' for better readability
my_data.fillna('0',inplace = True)
my_data.reset_index(inplace = True, drop=True)
# my_data.sort_index(axis=1) #Transpose?

end1 = time.time()


print ("Time elapsed:", end1 - start1)

Time elapsed: 13.606997966766357


In [39]:
# img = cv2.imread("screen0.png")
# count = 0

# for hold in holds:
#     cv2.circle(img,hold,3,(0,0,255),-1)
#     cv2.putText(img,f"{count}",hold,cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0))
#     count += 1
    
# cv2.imwrite("circles.png",img)

In [40]:
# my_data.style.format({'filepath': make_clickable})


In [41]:
# bad_data = my_data[~my_data['grade'].str.isdigit()]
# bad_data.style.format({'filepath': make_clickable})

In [42]:
# campus_data = my_data[my_data['campus']==True]
# campus_data.style.format({'filepath': make_clickable})

In [43]:
# my_data

In [44]:
my_data.to_csv('data.csv')
# my_data.to_csv('data_split.csv')